<div style="display: flex; align-items: center;">
    <img src="../figures/Mines Geophysics Black Moon Circle Waves 3.3.png" alt="Example Image" width="10%">
    <div style="margin-left: 10px;">
        <h1>GP100 Alumni Map</h1>
        <h2>Data Analysis</h2>
    </div>
</div>

### Loading the Data

In [1]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
# load data into a dataframe
path = '../data/GP_Alumni_List.csv'
df = pd.read_csv(path)

### Cleaning the Data

In [3]:
# clean the dataframe

#rename StateOrProvince to State
df.rename(columns={'StateOrProvince': 'State'}, inplace=True)

#delete rows if Affiliation is not alumni
df = df[df['Affiliation'] == 'Alumni']

#clean Degree column to include only degree level
validDegrees = ['BSc','MSc','PhD']
df['Degrees'] = df['Degrees'].fillna('').apply(lambda x: [deg.strip() for deg in x.split() if deg.strip() in validDegrees])

#split people with multiple degrees into separate rows
df = df.explode('Degrees')

#replace NaN in country column with United States or Canada based on StateorProvince
usStates = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE',
                                   'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS',
                                   'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
                                   'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
                                   'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
                                   'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
                                   'WI', 'WY','PR']
df['Country'] = df.apply(lambda row: 'United States' if row['State'] in usStates else row['Country'], axis=1)

#remove province from Canada
df['State'] = np.where(df['Country'] == 'Canada', '', df['State'])

df.reset_index(drop=True, inplace=True)
df

,Affiliation,PrefClassYear,Degrees,StateOrProvince,Country
0,Alumni,1983.0,BSc,CO,United States
1,Alumni,1995.0,BSc,CO,United States
2,Alumni,2013.0,MSc,NaN,Bahrain
3,Alumni,1999.0,MSc,NaN,United Kingdom
4,Alumni,1982.0,BSc,OK,United States
...,...,...,...,...,...
2225,Alumni,2012.0,BSc,CO,United States
2226,Alumni,1982.0,BSc,TX,United States
2227,Alumni,1981.0,BSc,UT,United States
2228,Alumni,2019.0,BSc,CO,United States


In [5]:
df.to_csv('../data/GP_Alumni_List_Cleaned.csv', index=False)

In [ ]:
#for the purposes of plotting, we need to assign each location a longitude and latitude
'''reference:
https://gis.stackexchange.com/questions/212796/getting-latlon-extent-of-country-by-its-name-using-python

def get_boundingbox_country(country, output_as='boundingbox'):
    """
    get the bounding box of a country in EPSG4326 given a country name

    Parameters
    ----------
    country : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search?country=',
                             country,
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output
'''

In [ ]:
def get_boundingbox_country(country, output_as='boundingbox'):
    """
    get the bounding box of a country in EPSG4326 given a country name

    Parameters
    ----------
    country : str
        name of the country in English and lowercase
    output_as : str
        choose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # Check if country name is not NaN
    if isinstance(country, str) or isinstance(country, np.float64):
        if isinstance(country, np.float64):
            country = str(country)  # Convert NaN float to string
        
        # Create URL
        url = 'http://nominatim.openstreetmap.org/search?country=' + country + '&format=json&polygon=0'
        
        try:
            response = requests.get(url).json()
            if response:
                if output_as == 'boundingbox':
                    lst = response[0][output_as]
                    output = [float(i) for i in lst]
                elif output_as == 'center':
                    lst = [response[0].get(key) for key in ['lat','lon']]
                    output = [float(i) for i in lst]
                else:
                    print("Error: Invalid output format specified")
                    return None
                return output
            else:
                print(f"Error: Unable to find coordinates for {country}")
                return None
        except Exception as e:
            print(f"Error: {e}")
            return None
    else:
        print(f"Error: Invalid country name: {country}")
        return None

# count unique states, provinces, and countries
uniqueStates = df['StateOrProvince'].unique()
uniqueCountries = df['Country'].unique()

# create a dictionary to store country centers
countryCenters = {}

# get latitude and longitude centers for each country
for country in uniqueCountries:
    center = get_boundingbox_country(country, output_as='center')
    if center:
        countryCenters[country] = center

#manually add some latitudes and longtitudes
countryCenters['Taiwan, Province of China'] = ['23.6978','120.9605']
countryCenters['Tanzania, United Republic Of'] = ['-6.3690','34.8888']

# add latitude and longitude centers to the df
df['Latitude'] = df['Country'].map(lambda x: countryCenters.get(x, [None, None])[0])
df['Longitude'] = df['Country'].map(lambda x: countryCenters.get(x, [None, None])[1])

# save updated df to GP_Alumni_List_Cleaned.csv
df.to_csv(path, index=False)